In [1]:
import pandas as pd
import numpy as np
import math as math
import statistics
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
import datetime
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
from sklearn.metrics import log_loss
from sklearn.preprocessing import StandardScaler
from mlxtend.plotting import plot_decision_regions
import time

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('max_colwidth', 1000)  #
pd.set_option('display.width', 150)

# from functions import *
#https://www.nba.com/stats/teams/boxscores-scoring

In [2]:
def convertCSVDateColumn(df):
    locationToInsert = df.columns.get_loc('GAME DATE')+1
    columnToInsert = pd.to_datetime(df['GAME DATE'],format="%m/%d/%y").dt.date
    df = df.drop(columns='GAME DATE')
    df.insert(locationToInsert,'GAME DATE',columnToInsert)
    df = df.sort_values(by=['GAME DATE','GAMEID']).reset_index(drop=True)
    return df
def featureEngineering(df):
    featuresToAddDict = {"OPPT":[],"PTSA":[],"2PA":[],"2PM":[],"2P%":[],"RESULT":[],"PTSDELTA":[],"H/A":[]}
    for i in range(len(df)):
        featuresToAddDict["OPPT"].append(df.iloc[i]['MATCH UP'][-3:])
        if i % 2 == 0: featuresToAddDict["PTSA"].append(df.iloc[i+1]['PTS'])
        else: featuresToAddDict["PTSA"].append(df.iloc[i-1]['PTS'])
        if df['W/L'][i] == 'L': featuresToAddDict["RESULT"].append(0)
        else: featuresToAddDict["RESULT"].append(1)
        featuresToAddDict["H/A"].append(df['MATCH UP'][i].count('vs'))
    df.insert(df.columns.get_loc('TEAM')+1,'OPPT',featuresToAddDict["OPPT"])
    df.insert(df.columns.get_loc('PTS')+1,'PTSA',featuresToAddDict["PTSA"])
    df.insert(df.columns.get_loc('FG%')+1,'2PM',df['FGM'] - df['3PM'])
    df.insert(df.columns.get_loc('FG%')+1,'2PA',df['FGA'] - df['3PA']) 
    df.insert(df.columns.get_loc('FG%')+1,'2P%',round(100*(df['2PM']/df['2PA']),2))
    df.insert(df.columns.get_loc('W/L')+1,'RESULT',featuresToAddDict["RESULT"])
    df = df.drop(columns=['W/L'])
    df.insert(df.columns.get_loc('PTSA')+1,'PTSDELTA',df['PTS']-df['PTSA'])
    df.insert(df.columns.get_loc('PTSDELTA')+1,'LR',np.log(df['PTS']/df['PTSA']))
    df.insert(df.columns.get_loc('GAME DATE')+1,'H/A',featuresToAddDict["H/A"])
    return df
def getDaysOff(teamsDict):
    for team_ in list(teamsDict.keys()):
        daysOff = []
        daysOff.append(np.nan)
        teamKey = team_
        for i in range(1,len(teamsDict[teamKey]['GAME DATE'])):
            date = teamsDict[teamKey]['GAME DATE'][i]
            prevDate = teamsDict[teamKey]['GAME DATE'][i-1] + datetime.timedelta(1)
            daysOff.append((date - prevDate).days)
        teamsDict[teamKey]['DAYSOFF'] = daysOff
def addAdditionalFeatures(df,teamsDict):
    teamDFA = []
    for team_ in list(teamsDict.keys()):
        teamDF = df[df['TEAM'] == team_]
        startingIndex = len(list(df.columns))
        additionalFeatures = list(teamsDict[team_].keys())[startingIndex:]
        for feature_ in additionalFeatures:
            teamDF = pd.concat([teamDF,pd.DataFrame(teamsDict[team_][feature_],
                                    columns=[feature_],
                                    index=teamDF.index.values)],axis=1)
        teamDFA.append(teamDF)
    return pd.concat(teamDFA)
def getDataFromDataFrame(df):
    teamsDict = {}
    for team_ in np.unique(df['TEAM']):
        teamsDict[team_] = {'TEAM':[],'OPPT':[],'TEAMID':[],'MATCH UP':[],'GAMEID':[],'RESULT':[],
                            'GAME DATE':[],'H/A':[],'SPREAD':[],'O/U':[],'MIN':[],'PTS':[],'PTSA':[],
                            'PTSDELTA':[],'LR':[],'FGM':[],'FGA':[],'FG%':[],'2P%':[],'2PA':[],'2PM':[],
                            '3PM':[],'3PA':[],'3P%':[],'FTM':[],'FTA':[],'FT%':[],'OREB':[],'DREB':[],
                            'REB':[],'AST':[],'STL':[],'BLK':[],'TOV':[],'PF':[],'+/-':[],
                            'Ws':[],'Ls':[],'GAMES PLAYED':[],'W%':[],'L%':[],'AVGPTS':[],'AVGPTSA':[]}
        features = df.columns.values
    for i in range(len(df)):
        for feature_ in features:
            teamKey = df.iloc[i]['TEAM']
            featureKey = feature_
            teamsDict[teamKey][featureKey].append(df.iloc[i][feature_])
        gamesPlayed = teamsDict[teamKey]['RESULT'].count(1) + teamsDict[teamKey]['RESULT'].count(0)
        teamsDict[teamKey]['Ws'].append(teamsDict[teamKey]['RESULT'].count(1))
        teamsDict[teamKey]['Ls'].append(teamsDict[teamKey]['RESULT'].count(0))
        teamsDict[teamKey]['GAMES PLAYED'].append(gamesPlayed)
        teamsDict[teamKey]['W%'].append(100*round(teamsDict[teamKey]['RESULT'].count(1)/gamesPlayed,3))
        teamsDict[teamKey]['L%'].append(100*round(teamsDict[teamKey]['RESULT'].count(0)/gamesPlayed,3))
        teamsDict[teamKey]["AVGPTS"].append(np.mean(teamsDict[teamKey]['PTS']))
        teamsDict[teamKey]["AVGPTSA"].append(np.mean(teamsDict[teamKey]['PTSA']))
    return teamsDict
def getMovingAverageLx(teamsDict,x):
    features = ['PTS','PTSA','PTSDELTA','LR','FGM','FGA','2PM','2PA','3PM','3PA','FTM','FTA',
                'OREB','DREB','REB','AST','TOV','STL','BLK','PF']
    MAFeatures = ['PTSLxMA','PTSALxMA','PTSDELTALxMA','LRLxMA','FGMLxMA','FGALxMA','2PMLxMA',
                  '2PALxMA','3PMLxMA','3PALxMA','FTMLxMA','FTALxMA','OREBLxMA','DREBLxMA',
                  'REBLxMA','ASTLxMA','TOVLxMA','STLLxMA','BLKLxMA','PFLxMA']
    for team_ in list(teamsDict.keys()):
        teamKey = team_
        for feature_ in features:
            ma = []
            for _ in range(x): ma.append(np.nan)
            for maRange in range(0,len(teamsDict[teamKey][feature_])-x):
                ma.append(np.mean(teamsDict[teamKey][feature_][maRange:(x+maRange)]))
            teamsDict[teamKey][MAFeatures[features.index(feature_)].replace('x',str(x))] = ma
def getPercentMovingAverageLx(teamsDict,x):
    features = ['FGM','FGA','2PM','2PA','3PM','3PA','FTM','FTA','Ws','GAMES PLAYED']
    MAfeatures = ['FG%LxMA','2P%LxMA','3P%LxMA','FT%LxMA','W%LxMA']
    for team_ in list(teamsDict.keys()):
        teamKey = team_
        for i in range(0,len(features),2):
            ma = []
            for _ in range(x): ma.append(np.nan)
            for maRange in range(0,len(teamsDict[teamKey][features[i]])-x):
                num = np.sum(teamsDict[teamKey][features[i]][maRange:(x+maRange)])
                dem = np.sum(teamsDict[teamKey][features[i+1]][maRange:(x+maRange)])
                ma.append(num/dem)
            teamsDict[teamKey][MAfeatures[int(i/2)].replace('x',str(x))] = ma

# Traditional.csv

In [3]:
start_time = time.time()
trainDF = pd.read_csv(
    '/Users/olutosinfashusi/jupyter/csv/fromNBADOTCOM/2023-24/byGame/Traditional.csv'
)
trainDF = convertCSVDateColumn(trainDF)
FE_trainDF = featureEngineering(trainDF)
dict_FE_trainDF = getDataFromDataFrame(FE_trainDF)
# getMovingAverageLx(dict_FE_trainDF,5)
# getPercentMovingAverageLx(dict_FE_trainDF,5)
# getMovingAverageLx(dict_FE_trainDF,4)
# getPercentMovingAverageLx(dict_FE_trainDF,4)
# getMovingAverageLx(dict_FE_trainDF,3)
# getPercentMovingAverageLx(dict_FE_trainDF,3)
# getMovingAverageLx(dict_FE_trainDF,2)
# getPercentMovingAverageLx(dict_FE_trainDF,2)
# getMovingAverageLx(dict_FE_trainDF,1)
# getPercentMovingAverageLx(dict_FE_trainDF,1)
# getDaysOff(dict_FE_trainDF)
AF_FE_trainDF = addAdditionalFeatures(FE_trainDF, dict_FE_trainDF)
print("%s seconds" % (time.time() - start_time))

35.56104898452759 seconds


In [4]:
NBAdf = AF_FE_trainDF.sort_values(by=['GAME DATE', 'GAMEID', 'H/A']).reset_index(drop=True)
NBAteams = np.unique(NBAdf['TEAM'])

In [45]:
NBAdf[NBAdf['TEAM'] == 'CHA']

,TEAM,OPPT,TEAMID,MATCH UP,GAMEID,RESULT,GAME DATE,H/A,SPREAD,O/U,MIN,PTS,PTSA,PTSDELTA,LR,FGM,FGA,FG%,2P%,2PA,2PM,3PM,3PA,3P%,FTM,FTA,FT%,OREB,DREB,REB,AST,STL,BLK,TOV,PF,+/-,Ws,Ls,GAMES PLAYED,W%,L%,AVGPTS,AVGPTSA
5,CHA,ATL,1610612766,CHA vs. ATL,22300063,1,2023-10-25,1,4.0,236.0,240,116,110,6,0.053110,43,86,50.0,65.31,49,32,11,37,29.7,19,26,73.1,12,39,51,34,5,3,19,21,6,1,0,1,100.0,0.0,116.000000,110.000000
33,CHA,DET,1610612766,CHA vs. DET,22300077,0,2023-10-27,1,-4.5,227.0,240,99,111,-12,-0.114410,33,88,37.5,43.33,60,26,7,28,25.0,26,34,76.5,11,30,41,20,13,7,18,23,-12,1,1,2,50.0,50.0,107.500000,110.500000
81,CHA,BKN,1610612766,CHA vs. BKN,22300101,0,2023-10-30,1,1.0,228.5,240,121,133,-12,-0.094559,47,96,49.0,60.00,65,39,8,31,25.8,19,20,95.0,8,31,39,33,8,5,10,23,-12,1,2,3,33.3,66.7,112.000000,118.000000
122,CHA,HOU,1610612766,CHA @ HOU,22300122,0,2023-11-01,0,3.0,222.0,240,119,128,-9,-0.072907,45,90,50.0,53.97,63,34,11,27,40.7,18,25,72.0,15,29,44,21,9,8,16,19,-9,1,3,4,25.0,75.0,113.750000,120.500000
158,CHA,IND,1610612766,CHA @ IND,22300133,1,2023-11-04,0,5.0,241.5,240,125,124,1,0.008032,47,88,53.4,57.35,68,39,8,20,40.0,23,23,100.0,12,29,41,23,5,10,13,20,1,2,3,5,40.0,60.0,116.000000,121.200000
178,CHA,DAL,1610612766,CHA @ DAL,22300143,0,2023-11-05,0,11.5,235.0,240,118,124,-6,-0.049597,49,99,49.5,59.42,69,41,8,30,26.7,12,21,57.1,10,33,43,30,5,2,9,25,-6,2,4,6,33.3,66.7,116.333333,121.666667
207,CHA,WAS,1610612766,CHA vs. WAS,22300157,0,2023-11-08,1,-2.0,239.0,240,116,132,-16,-0.129212,41,79,51.9,61.11,54,33,8,25,32.0,26,35,74.3,11,36,47,17,8,5,22,22,-16,2,5,7,28.6,71.4,116.285714,123.142857
240,CHA,WAS,1610612766,CHA @ WAS,22300009,1,2023-11-10,0,3.5,240.5,240,124,117,7,0.058108,50,110,45.5,53.66,82,44,6,28,21.4,18,28,64.3,28,39,67,31,10,4,10,19,7,3,5,8,37.5,62.5,117.250000,122.375000
264,CHA,NYK,1610612766,CHA @ NYK,22300177,0,2023-11-12,0,10.5,224.0,240,107,129,-22,-0.186984,44,94,46.8,54.24,59,32,12,35,34.3,7,11,63.6,15,21,36,21,5,4,12,25,-22,3,6,9,33.3,66.7,116.111111,123.111111
295,CHA,MIA,1610612766,CHA vs. MIA,22300017,0,2023-11-14,1,6.5,221.5,240,105,111,-6,-0.055570,36,81,44.4,42.86,49,21,15,32,46.9,18,22,81.8,13,32,45,21,6,0,18,21,-6,3,7,10,30.0,70.0,115.000000,121.900000


In [5]:
# dict_ = {}
# for team in NBAteams:
#     dict_[team] = {}
#     dict_[team]['FAV#'] = int
#     dict_[team]['✅'] = int
#     dict_[team]['❌'] = int

In [6]:
dict_ = {}
for team in NBAteams:
    dict_[team] = {}

for team in NBAteams:
    teamDF = NBAdf[NBAdf['TEAM'] == team]
    teamDF = teamDF[teamDF['SPREAD'] < 0]
    dict_[team]['#FAV'] = len(teamDF[teamDF['SPREAD'] < 0])
    dict_[team]['BEAT SPEAD'] = sum(abs(teamDF['SPREAD']) < teamDF['PTSDELTA'])
    dict_[team]['BEAT SPEAD%'] = dict_[team]['BEAT SPEAD'] / dict_[team]['#FAV']
    dict_[team]['LOST SPREAD'] = sum(abs(teamDF['SPREAD']) > teamDF['PTSDELTA'])
    dict_[team]['LOST SPREAD%'] = dict_[team]['LOST SPREAD'] / dict_[team]['#FAV']
    dict_[team]['PUSH'] = sum(abs(teamDF['SPREAD']) ==  teamDF['PTSDELTA'])
    dict_[team]['PUSH%'] = dict_[team]['PUSH'] / dict_[team]['#FAV']
    
# for team in NBAteams:
#     teamDF = NBAdf[NBAdf['TEAM'] == team]
#     teamDF = teamDF[teamDF['SPREAD'] < 0]
#     dict_[team]['#FAV'] = len(teamDF[teamDF['SPREAD'] < 0])
#     dict_[team]['BEAT SPREAD'] = 0
#     dict_[team]['LOST SPREAD'] = 0
#     dict_[team]['PUSH'] = 0
#     for i in range(len(teamDF)):
#         if teamDF.iloc[i]['PTSDELTA'] < 0: #lost
#             dict_[team]['LOST SPREAD'] = dict_[team]['LOST SPREAD'] + 1
#         if teamDF.iloc[i]['PTSDELTA'] > 0: #won
#             if np.abs(teamDF.iloc[i]['SPREAD']) > teamDF.iloc[i]['PTSDELTA']:
#                 dict_[team]['LOST SPREAD'] = dict_[team]['LOST SPREAD'] + 1
#             if np.abs(teamDF.iloc[i]['SPREAD']) == teamDF.iloc[i]['PTSDELTA']:
#                 dict_[team]['PUSH'] = dict_[team]['PUSH'] + 1
#             if np.abs(teamDF.iloc[i]['SPREAD']) < teamDF.iloc[i]['PTSDELTA']:
#                 dict_[team]['BEAT SPREAD'] = dict_[team]['BEAT SPREAD'] + 1
spreadResultsDF = pd.DataFrame(dict_).T

In [7]:
# teamDF = NBAdf[NBAdf['TEAM'] == 'ATL']
# teamDF = teamDF[teamDF['SPREAD'] < 0]
# teamDF.reset_index(drop=True).head()

In [8]:
# abs(teamDF.iloc[2]['SPREAD']) > teamDF.iloc[2]['PTSDELTA']

In [9]:
spreadResultsDF.head()

,#FAV,BEAT SPEAD,BEAT SPEAD%,LOST SPREAD,LOST SPREAD%,PUSH,PUSH%
ATL,43.0,15.0,0.348837,27.0,0.627907,1.0,0.023256
BKN,26.0,13.0,0.500000,12.0,0.461538,1.0,0.038462
BOS,78.0,40.0,0.512821,33.0,0.423077,5.0,0.064103
CHA,7.0,1.0,0.142857,6.0,0.857143,0.0,0.000000
CHI,33.0,15.0,0.454545,17.0,0.515152,1.0,0.030303


In [10]:
dict_ = {}
for team in NBAteams:
    dict_[team] = {}
    dict_[team]['gamesFav+LostSpread'] = pd.DataFrame
    
for team in NBAteams:
    teamDF = NBAdf[NBAdf['TEAM'] == team]
    teamDF = teamDF[teamDF['SPREAD'] < 0]
    teamDF = teamDF.reset_index(drop=True)

    df = pd.DataFrame()
    for i in range(len(teamDF)):
        if teamDF.iloc[i]['PTSDELTA'] < 0:
            df = pd.concat([df,teamDF.iloc[i].to_frame().T],axis=0)
        if teamDF.iloc[i]['PTSDELTA'] > 0:
            if np.abs(teamDF.iloc[i]['SPREAD']) > teamDF.iloc[i]['PTSDELTA']:
                df = pd.concat([df,teamDF.iloc[i].to_frame().T],axis=0)
    dict_[team]['gamesFav+LostSpread'] = df.reset_index(drop=True)

In [12]:
resultsDict = {}
for team in NBAteams:
    resultsDict[team] = []

In [13]:
def cleanBoxScore(boxScoreDF):
    if 'Unnamed: 22' in boxScoreDF.columns:
            boxScoreDF = boxScoreDF.drop(columns='Unnamed: 22')
    boxScoreDF['PLAYER']
    players = []
    for playerIter in range(5):
        players.append(boxScoreDF['PLAYER'].iloc[playerIter][:-1])
    for playerIter in range(5,len(boxScoreDF)):
        players.append(boxScoreDF['PLAYER'].iloc[playerIter])
    boxScoreDF = boxScoreDF.drop(columns='PLAYER')
    boxScoreDF.insert(0,'PLAYER',players)
    boxScoreDF = boxScoreDF.reset_index(drop=True).dropna()
    
    rowsToDrop = []
    for i in range(len(boxScoreDF)):
        if 'DNP' in boxScoreDF.iloc[i]['MIN'] or ' ' in boxScoreDF.iloc[i]['MIN'] or 'SUSPENSION' in boxScoreDF.iloc[i]['MIN']:
            rowsToDrop.append(i)

    boxScoreDF = boxScoreDF.drop(rowsToDrop)
    boxScoreDF['PTS_'] = boxScoreDF['PTS'].astype(int)
    boxScoreDF = boxScoreDF.drop(columns='PTS')
    boxScoreDF = boxScoreDF.rename(columns={'PTS_':'PTS'})
    return boxScoreDF

In [14]:
results = []
for team in NBAteams:
    print(team)
    dict_[team]['gamesFav+LostSpread'].head()
    teamDF = dict_[team]['gamesFav+LostSpread']
    dateOfFirstGame = NBAdf[NBAdf['TEAM'] == team].head(1)['GAME DATE'].values[0]
    teamDF = teamDF[teamDF['GAME DATE'] > dateOfFirstGame]
    teamDF = teamDF.reset_index(drop=True);
    
    for i in range (len(teamDF)):
        GAMEID = teamDF.iloc[i]['GAMEID']
        GAMEID = str(GAMEID)
        #print(GAMEID,teamDF.iloc[i]['GAME DATE'])
        boxScoreDF = pd.read_csv('/Users/olutosinfashusi/jupyter/csv/boxScoreCSV/boxScoreCSVs/'+ GAMEID +'.csv')#.fillna(' ')
        boxScoreDF = boxScoreDF[boxScoreDF['TEAM'] ==  team]
        boxScoreDF.insert(1,'OPPT',teamDF.iloc[i]['OPPT'])
        boxScoreDF = boxScoreDF.reset_index(drop=True).dropna()
        boxScoreDF = cleanBoxScore(boxScoreDF);

        prevGamesDF = NBAdf[NBAdf['TEAM'] == team]
        prevGamesDF  = prevGamesDF[prevGamesDF['GAME DATE'] < teamDF.iloc[i]['GAME DATE']]
        prevGamesDF.reset_index(drop=True);

        prevGameIDs = prevGamesDF['GAMEID'].values
        prevOppts = prevGamesDF['OPPT'].values

        prevBoxScoresDF = pd.DataFrame()
        for prevIter in range (len(prevGameIDs)):
            GAMEID = str(prevGameIDs[prevIter])
            #print('PREV:',GAMEID)
            prevBoxScoreDF = pd.read_csv('/Users/olutosinfashusi/jupyter/csv/boxScoreCSV/boxScoreCSVs/'+ GAMEID +'.csv')#.fillna(' ')
            prevBoxScoreDF = prevBoxScoreDF[prevBoxScoreDF['TEAM'] == team]
            prevBoxScoreDF.insert(1,'OPPT',prevOppts[prevIter])
            prevBoxScoreDF = cleanBoxScore(prevBoxScoreDF)
            prevBoxScoresDF = pd.concat([prevBoxScoresDF,prevBoxScoreDF],axis=0)

        prevBoxScoresDF.groupby("PLAYER")['PTS'].mean()
        boxScoreDF.groupby("PLAYER")['PTS'].mean()
        prevAVGPTS = pd.DataFrame(prevBoxScoresDF.groupby("PLAYER")['PTS'].mean())

        prevAVGPTS = prevAVGPTS.rename(columns={'PTS':'PREV AVG PTS'})
        df = pd.concat([boxScoreDF.groupby("PLAYER")['PTS'].mean(),
               prevAVGPTS.groupby("PLAYER")['PREV AVG PTS'].mean()],axis=1).dropna()
        df['DATE'] = teamDF.iloc[i]['GAME DATE']
        df['TEAM'] = teamDF.iloc[i]['TEAM']
        df['OPPT'] = teamDF.iloc[i]['OPPT']
        df['PTSDELTA'] = teamDF.iloc[i]['PTSDELTA']
        df['SPREAD'] = teamDF.iloc[i]['SPREAD']
        resultsDict[team].append(df)

ATL
BKN
BOS
CHA
CHI
CLE
DAL
DEN
DET
GSW
HOU
IND
LAC
LAL
MEM
MIA
MIL
MIN
NOP
NYK
OKC
ORL
PHI
PHX
POR
SAC
SAS
TOR
UTA
WAS


In [15]:
team = 'ATL'
FLAG = []
resultsDF = pd.DataFrame()
for i in range (len(resultsDict[team])):
    resultsDF = pd.concat([resultsDF,pd.DataFrame(resultsDict[team][i])],axis=0)
for i in range(len(resultsDF)):
    if 10+resultsDF.iloc[i]['PTS'] < resultsDF.iloc[i]['PREV AVG PTS']:
        FLAG.append('🚩')
    else: FLAG.append(' ')
resultsDF['FLAG'] = FLAG
resultsDF[resultsDF['FLAG'] == '🚩'].sort_values(by='PLAYER')

,PTS,PREV AVG PTS,DATE,TEAM,OPPT,PTSDELTA,SPREAD,FLAG
PLAYER,,,,,,,,
Jalen Johnson,5.0,15.434783,2024-01-13,ATL,WAS,-28,-8.0,🚩
Saddiq Bey,2.0,12.142857,2023-11-09,ATL,ORL,1,-3.5,🚩
Trae Young,13.0,27.823529,2024-01-12,ATL,IND,-18,-6.0,🚩
Trae Young,12.0,26.959184,2024-02-14,ATL,CHA,-23,-7.0,🚩
Trae Young,11.0,26.660000,2024-02-23,ATL,TOR,-2,-7.5,🚩
Trae Young,14.0,26.352941,2024-04-10,ATL,CHA,-1,-8.5,🚩


In [16]:
team = 'BKN'
FLAG = []
resultsDF = pd.DataFrame()
for i in range (len(resultsDict[team])):
    resultsDF = pd.concat([resultsDF,pd.DataFrame(resultsDict[team][i])],axis=0)
for i in range(len(resultsDF)):
    if 10+resultsDF.iloc[i]['PTS'] < resultsDF.iloc[i]['PREV AVG PTS']:
        FLAG.append('🚩')
    else: FLAG.append(' ')
resultsDF['FLAG'] = FLAG
resultsDF[resultsDF['FLAG'] == '🚩'].sort_values(by='PLAYER')

,PTS,PREV AVG PTS,DATE,TEAM,OPPT,PTSDELTA,SPREAD,FLAG
PLAYER,,,,,,,,
Cam Thomas,9.0,20.740741,2024-01-07,BKN,POR,-7,-9.0,🚩
Cam Thomas,8.0,20.600000,2024-01-17,BKN,POR,-2,-5.0,🚩
Lonnie Walker IV,0.0,14.000000,2024-01-07,BKN,POR,-7,-9.0,🚩
Lonnie Walker IV,0.0,12.040000,2024-01-27,BKN,HOU,2,-4.0,🚩


In [17]:
team = 'BOS'
FLAG = []
resultsDF = pd.DataFrame()
for i in range (len(resultsDict[team])):
    resultsDF = pd.concat([resultsDF,pd.DataFrame(resultsDict[team][i])],axis=0)
for i in range(len(resultsDF)):
    if 10+resultsDF.iloc[i]['PTS'] < resultsDF.iloc[i]['PREV AVG PTS']:
        FLAG.append('🚩')
    else: FLAG.append(' ')
resultsDF['FLAG'] = FLAG
resultsDF[resultsDF['FLAG'] == '🚩'].sort_values(by='PLAYER')

,PTS,PREV AVG PTS,DATE,TEAM,OPPT,PTSDELTA,SPREAD,FLAG
PLAYER,,,,,,,,
Derrick White,2.0,16.000000,2024-01-27,BOS,LAC,-19,-7.0,🚩
Derrick White,3.0,15.545455,2024-03-05,BOS,CLE,-1,-8.5,🚩
Derrick White,4.0,15.826087,2024-02-09,BOS,WAS,4,-18.0,🚩
Derrick White,2.0,15.380282,2024-04-11,BOS,NYK,-9,-3.5,🚩
Jaylen Brown,12.0,22.909091,2023-11-19,BOS,MEM,2,-12.0,🚩
Jaylen Brown,8.0,22.536585,2024-01-27,BOS,LAC,-19,-7.0,🚩
Jaylen Brown,8.0,22.250000,2024-02-01,BOS,LAL,-9,-15.5,🚩
Jaylen Brown,11.0,23.166667,2023-11-08,BOS,PHI,-3,-2.5,🚩
Jayson Tatum,17.0,28.454545,2023-11-17,BOS,TOR,3,-9.0,🚩


In [18]:
team = 'CHA'
FLAG = []
resultsDF = pd.DataFrame()
for i in range (len(resultsDict[team])):
    resultsDF = pd.concat([resultsDF,pd.DataFrame(resultsDict[team][i])],axis=0)
for i in range(len(resultsDF)):
    if 10+resultsDF.iloc[i]['PTS'] < resultsDF.iloc[i]['PREV AVG PTS']:
        FLAG.append('🚩')
    else: FLAG.append(' ')
resultsDF['FLAG'] = FLAG
resultsDF[resultsDF['FLAG'] == '🚩'].sort_values(by='PLAYER')

,PTS,PREV AVG PTS,DATE,TEAM,OPPT,PTSDELTA,SPREAD,FLAG
PLAYER,,,,,,,,
Gordon Hayward,4.0,15.083333,2023-11-22,CHA,WAS,3,-3.5,🚩
Mark Williams,2.0,13.000000,2023-10-27,CHA,DET,-12,-4.5,🚩
Miles Bridges,9.0,20.461538,2023-12-16,CHA,PHI,-53,-12.0,🚩
P.J. Washington,13.0,25.000000,2023-10-27,CHA,DET,-12,-4.5,🚩
Terry Rozier,6.0,24.357143,2023-12-16,CHA,PHI,-53,-12.0,🚩


In [19]:
team = 'CLE'
FLAG = []
resultsDF = pd.DataFrame()
for i in range (len(resultsDict[team])):
    resultsDF = pd.concat([resultsDF,pd.DataFrame(resultsDict[team][i])],axis=0)
for i in range(len(resultsDF)):
    if 10+resultsDF.iloc[i]['PTS'] < resultsDF.iloc[i]['PREV AVG PTS']:
        FLAG.append('🚩')
    else: FLAG.append(' ')
resultsDF['FLAG'] = FLAG
resultsDF[resultsDF['FLAG'] == '🚩'].sort_values(by='PLAYER')

,PTS,PREV AVG PTS,DATE,TEAM,OPPT,PTSDELTA,SPREAD,FLAG
PLAYER,,,,,,,,
Caris LeVert,6.0,17.500000,2023-11-25,CLE,LAL,-6,-4.0,🚩
Caris LeVert,5.0,15.533333,2023-11-30,CLE,POR,-8,-12.5,🚩
Caris LeVert,4.0,15.342105,2024-02-07,CLE,WAS,8,-11.5,🚩
Darius Garland,6.0,20.300000,2023-11-25,CLE,LAL,-6,-4.0,🚩
Darius Garland,9.0,20.619048,2024-02-01,CLE,MEM,7,-8.5,🚩
Darius Garland,8.0,18.218750,2024-02-27,CLE,DAL,2,-4.0,🚩
Donovan Mitchell,20.0,32.500000,2023-11-08,CLE,OKC,-8,-2.0,🚩
Donovan Mitchell,10.0,28.545455,2023-11-26,CLE,TOR,3,-3.5,🚩
Donovan Mitchell,17.0,27.615385,2024-01-07,CLE,SAS,2,-10.5,🚩


In [20]:
team = 'DAL'
FLAG = []
resultsDF = pd.DataFrame()
for i in range (len(resultsDict[team])):
    resultsDF = pd.concat([resultsDF,pd.DataFrame(resultsDict[team][i])],axis=0)
for i in range(len(resultsDF)):
    if 10+resultsDF.iloc[i]['PTS'] < resultsDF.iloc[i]['PREV AVG PTS']:
        FLAG.append('🚩')
    else: FLAG.append(' ')
resultsDF['FLAG'] = FLAG
resultsDF[resultsDF['FLAG'] == '🚩'].sort_values(by='PLAYER')

,PTS,PREV AVG PTS,DATE,TEAM,OPPT,PTSDELTA,SPREAD,FLAG
PLAYER,,,,,,,,
Derrick Jones Jr.,0.0,10.200000,2023-11-05,DAL,CHA,6,-11.5,🚩
Grant Williams,5.0,17.000000,2023-10-27,DAL,BKN,5,-6.0,🚩
Kyrie Irving,10.0,24.571429,2023-12-01,DAL,MEM,-14,-6.0,🚩
Luka Doncic,23.0,33.800000,2023-11-05,DAL,CHA,6,-11.5,🚩
Luka Doncic,16.0,32.600000,2023-11-14,DAL,NOP,-21,-3.5,🚩
Luka Doncic,19.0,33.900000,2024-01-01,DAL,UTA,-37,-4.5,🚩
Luka Doncic,18.0,34.372881,2024-03-19,DAL,SAS,6,-9.0,🚩
Tim Hardaway Jr.,6.0,17.900000,2024-01-01,DAL,UTA,-37,-4.5,🚩
Tim Hardaway Jr.,5.0,17.720000,2024-02-12,DAL,WAS,8,-12.0,🚩


In [21]:
team = 'DEN'
FLAG = []
resultsDF = pd.DataFrame()
for i in range (len(resultsDict[team])):
    resultsDF = pd.concat([resultsDF,pd.DataFrame(resultsDict[team][i])],axis=0)
for i in range(len(resultsDF)):
    if 10+resultsDF.iloc[i]['PTS'] < resultsDF.iloc[i]['PREV AVG PTS']:
        FLAG.append('🚩')
    else: FLAG.append(' ')
resultsDF['FLAG'] = FLAG
resultsDF[resultsDF['FLAG'] == '🚩'].sort_values(by='PLAYER')

,PTS,PREV AVG PTS,DATE,TEAM,OPPT,PTSDELTA,SPREAD,FLAG
PLAYER,,,,,,,,
Aaron Gordon,2.0,13.866667,2023-11-24,DEN,HOU,-19,-3.0,🚩
Aaron Gordon,3.0,13.055556,2023-12-08,DEN,HOU,-8,-9.0,🚩
Jamal Murray,9.0,19.947368,2023-12-29,DEN,OKC,-26,-2.0,🚩
Jamal Murray,6.0,21.142857,2024-02-29,DEN,MIA,6,-6.5,🚩
Kentavious Caldwell-Pope,8.0,20.000000,2023-10-27,DEN,MEM,4,-6.0,🚩
Kentavious Caldwell-Pope,0.0,10.060606,2024-03-29,DEN,MIN,-13,-3.0,🚩
Michael Porter Jr.,5.0,16.289474,2024-01-10,DEN,UTA,-13,-6.5,🚩
Nikola Jokic,18.0,29.666667,2023-11-19,DEN,CLE,-12,-4.5,🚩
Nikola Jokic,9.0,28.769231,2023-11-20,DEN,DET,4,-9.5,🚩


In [22]:
team = 'DET'
FLAG = []
resultsDF = pd.DataFrame()
for i in range (len(resultsDict[team])):
    resultsDF = pd.concat([resultsDF,pd.DataFrame(resultsDict[team][i])],axis=0)
for i in range(len(resultsDF)):
    if 10+resultsDF.iloc[i]['PTS'] < resultsDF.iloc[i]['PREV AVG PTS']:
        FLAG.append('🚩')
    else: FLAG.append(' ')
resultsDF['FLAG'] = FLAG
resultsDF[resultsDF['FLAG'] == '🚩'].sort_values(by='PLAYER')

,PTS,PREV AVG PTS,DATE,TEAM,OPPT,PTSDELTA,SPREAD,FLAG
PLAYER,,,,,,,,
Bojan Bogdanovic,8.0,21.125000,2023-12-21,DET,UTA,-8,-2.0,🚩
Jalen Duren,0.0,13.892857,2024-04-01,DET,MEM,-2,-3.0,🚩


In [23]:
team = 'GSW'
FLAG = []
resultsDF = pd.DataFrame()
for i in range (len(resultsDict[team])):
    resultsDF = pd.concat([resultsDF,pd.DataFrame(resultsDict[team][i])],axis=0)
for i in range(len(resultsDF)):
    if 10+resultsDF.iloc[i]['PTS'] < resultsDF.iloc[i]['PREV AVG PTS']:
        FLAG.append('🚩')
    else: FLAG.append(' ')
resultsDF['FLAG'] = FLAG
resultsDF[resultsDF['FLAG'] == '🚩'].sort_values(by='PLAYER')

,PTS,PREV AVG PTS,DATE,TEAM,OPPT,PTSDELTA,SPREAD,FLAG
PLAYER,,,,,,,,
Jonathan Kuminga,0.0,16.222222,2024-04-14,GSW,UTA,7,-11.0,🚩
Klay Thompson,3.0,17.166667,2023-12-30,GSW,DAL,-10,-4.0,🚩
Stephen Curry,21.0,33.500000,2023-11-01,GSW,SAC,1,-6.5,🚩
Stephen Curry,7.0,28.956522,2023-12-17,GSW,POR,4,-6.0,🚩
Stephen Curry,13.0,27.892857,2023-12-28,GSW,MIA,-12,-6.5,🚩
Stephen Curry,9.0,27.606061,2024-01-07,GSW,TOR,-15,-2.5,🚩
Stephen Curry,15.0,27.175439,2024-03-07,GSW,CHI,-3,-8.5,🚩


In [24]:
team = 'HOU'
FLAG = []
resultsDF = pd.DataFrame()
for i in range (len(resultsDict[team])):
    resultsDF = pd.concat([resultsDF,pd.DataFrame(resultsDict[team][i])],axis=0)
for i in range(len(resultsDF)):
    if 10+resultsDF.iloc[i]['PTS'] < resultsDF.iloc[i]['PREV AVG PTS']:
        FLAG.append('🚩')
    else: FLAG.append(' ')
resultsDF['FLAG'] = FLAG
resultsDF[resultsDF['FLAG'] == '🚩'].sort_values(by='PLAYER')

,PTS,PREV AVG PTS,DATE,TEAM,OPPT,PTSDELTA,SPREAD,FLAG
PLAYER,,,,,,,,
Alperen Sengun,10.0,21.934783,2024-01-31,HOU,NOP,-11,-1.0,🚩
Jabari Smith Jr.,0.0,13.756098,2024-01-31,HOU,NOP,-11,-1.0,🚩
Jalen Green,5.0,17.111111,2023-12-26,HOU,IND,-6,-3.0,🚩
Jalen Green,7.0,17.375000,2024-01-20,HOU,UTA,1,-2.0,🚩
Jalen Green,4.0,18.264151,2024-02-14,HOU,MEM,-8,-2.5,🚩
Jalen Green,3.0,19.848101,2024-04-11,HOU,UTA,-3,-11.5,🚩


In [25]:
team = 'IND'
FLAG = []
resultsDF = pd.DataFrame()
for i in range (len(resultsDict[team])):
    resultsDF = pd.concat([resultsDF,pd.DataFrame(resultsDict[team][i])],axis=0)
for i in range(len(resultsDF)):
    if 10+resultsDF.iloc[i]['PTS'] < resultsDF.iloc[i]['PREV AVG PTS']:
        FLAG.append('🚩')
    else: FLAG.append(' ')
resultsDF['FLAG'] = FLAG
resultsDF[resultsDF['FLAG'] == '🚩'].sort_values(by='PLAYER')

,PTS,PREV AVG PTS,DATE,TEAM,OPPT,PTSDELTA,SPREAD,FLAG
PLAYER,,,,,,,,
Aaron Nesmith,5.0,16.500000,2023-10-30,IND,CHI,-7,-3.5,🚩
Aaron Nesmith,0.0,11.181818,2023-11-19,IND,ORL,-12,-4.5,🚩
Bennedict Mathurin,2.0,13.925926,2023-12-23,IND,ORL,-7,-2.5,🚩
Bennedict Mathurin,2.0,14.525000,2024-01-19,IND,POR,-3,-8.5,🚩
Bennedict Mathurin,0.0,14.120000,2024-02-12,IND,CHA,-9,-10.0,🚩
Myles Turner,6.0,17.590909,2023-12-15,IND,WAS,-14,-8.0,🚩
Obi Toppin,0.0,11.475000,2024-01-19,IND,POR,-3,-8.5,🚩
Tyrese Haliburton,12.0,24.700000,2023-11-19,IND,ORL,-12,-4.5,🚩
Tyrese Haliburton,5.0,22.921053,2024-02-08,IND,GSW,-22,-6.0,🚩


In [26]:
team = 'LAC'
FLAG = []
resultsDF = pd.DataFrame()
for i in range (len(resultsDict[team])):
    resultsDF = pd.concat([resultsDF,pd.DataFrame(resultsDict[team][i])],axis=0)
for i in range(len(resultsDF)):
    if 10+resultsDF.iloc[i]['PTS'] < resultsDF.iloc[i]['PREV AVG PTS']:
        FLAG.append('🚩')
    else: FLAG.append(' ')
resultsDF['FLAG'] = FLAG
resultsDF[resultsDF['FLAG'] == '🚩'].sort_values(by='PLAYER')

,PTS,PREV AVG PTS,DATE,TEAM,OPPT,PTSDELTA,SPREAD,FLAG
PLAYER,,,,,,,,
Ivica Zubac,6.0,20.000000,2023-10-27,LAC,UTA,-2,-4.0,🚩
James Harden,4.0,17.243902,2024-02-02,LAC,DET,11,-12.0,🚩
James Harden,6.0,16.984848,2024-03-31,LAC,CHA,12,-14.0,🚩
James Harden,4.0,16.732394,2024-04-12,LAC,UTA,-1,-15.5,🚩
Kawhi Leonard,6.0,24.000000,2024-03-12,LAC,MIN,-18,-5.5,🚩
Norman Powell,2.0,13.510204,2024-02-10,LAC,DET,6,-17.0,🚩
Paul George,6.0,25.214286,2023-11-27,LAC,DEN,-9,-11.5,🚩
Paul George,13.0,23.146341,2024-01-29,LAC,CLE,-10,-1.5,🚩
Paul George,7.0,22.511111,2024-02-07,LAC,NOP,-11,-5.5,🚩


In [27]:
team = 'LAL'
FLAG = []
resultsDF = pd.DataFrame()
for i in range (len(resultsDict[team])):
    resultsDF = pd.concat([resultsDF,pd.DataFrame(resultsDict[team][i])],axis=0)
for i in range(len(resultsDF)):
    if 10+resultsDF.iloc[i]['PTS'] < resultsDF.iloc[i]['PREV AVG PTS']:
        FLAG.append('🚩')
    else: FLAG.append(' ')
resultsDF['FLAG'] = FLAG
resultsDF[resultsDF['FLAG'] == '🚩'].sort_values(by='PLAYER')

,PTS,PREV AVG PTS,DATE,TEAM,OPPT,PTSDELTA,SPREAD,FLAG
PLAYER,,,,,,,,
Anthony Davis,9.0,23.400000,2023-11-15,LAL,SAC,-15,-1.0,🚩
Anthony Davis,14.0,24.949153,2024-03-06,LAL,SAC,-10,-3.0,🚩
Anthony Davis,8.0,24.714286,2024-03-16,LAL,GSW,-7,-1.5,🚩
D'Angelo Russell,4.0,18.384615,2023-11-19,LAL,HOU,1,-6.5,🚩
D'Angelo Russell,2.0,15.769231,2023-12-20,LAL,CHI,-16,-3.5,🚩
D'Angelo Russell,6.0,18.032787,2024-03-13,LAL,SAC,-13,-2.5,🚩
D'Angelo Russell,4.0,18.189189,2024-04-12,LAL,MEM,3,-16.0,🚩
LeBron James,12.0,25.419355,2024-01-03,LAL,MIA,-14,-4.5,🚩
Rui Hachimura,0.0,11.607143,2024-01-19,LAL,BKN,-18,-7.0,🚩


In [28]:
team = 'MEM'
FLAG = []
resultsDF = pd.DataFrame()
for i in range (len(resultsDict[team])):
    resultsDF = pd.concat([resultsDF,pd.DataFrame(resultsDict[team][i])],axis=0)
for i in range(len(resultsDF)):
    if 10+resultsDF.iloc[i]['PTS'] < resultsDF.iloc[i]['PREV AVG PTS']:
        FLAG.append('🚩')
    else: FLAG.append(' ')
resultsDF['FLAG'] = FLAG
resultsDF[resultsDF['FLAG'] == '🚩'].sort_values(by='PLAYER')

,PTS,PREV AVG PTS,DATE,TEAM,OPPT,PTSDELTA,SPREAD,FLAG
PLAYER,,,,,,,,
Jaren Jackson Jr.,10.0,21.03125,2024-01-02,MEM,SAS,8,-11.0,🚩


In [29]:
team = 'MIA'
FLAG = []
resultsDF = pd.DataFrame()
for i in range (len(resultsDict[team])):
    resultsDF = pd.concat([resultsDF,pd.DataFrame(resultsDict[team][i])],axis=0)
for i in range(len(resultsDF)):
    if 10+resultsDF.iloc[i]['PTS'] < resultsDF.iloc[i]['PREV AVG PTS']:
        FLAG.append('🚩')
    else: FLAG.append(' ')
resultsDF['FLAG'] = FLAG
resultsDF[resultsDF['FLAG'] == '🚩'].sort_values(by='PLAYER')

,PTS,PREV AVG PTS,DATE,TEAM,OPPT,PTSDELTA,SPREAD,FLAG
PLAYER,,,,,,,,
Bam Adebayo,8.0,21.027778,2024-01-29,MIA,PHX,-13,-2.5,🚩
Caleb Martin,0.0,10.461538,2024-03-22,MIA,NOP,-23,-3.0,🚩
Duncan Robinson,0.0,13.500000,2024-01-29,MIA,PHX,-13,-2.5,🚩
Duncan Robinson,2.0,13.276923,2024-04-04,MIA,PHI,-4,-2.5,🚩
Jaime Jaquez Jr.,2.0,13.825000,2024-01-29,MIA,PHX,-13,-2.5,🚩
Jimmy Butler,8.0,21.521739,2023-12-30,MIA,UTA,-8,-3.0,🚩
Kevin Love,0.0,10.129032,2024-01-19,MIA,ATL,-1,-7.0,🚩


In [30]:
team = 'MIL'
FLAG = []
resultsDF = pd.DataFrame()
for i in range (len(resultsDict[team])):
    resultsDF = pd.concat([resultsDF,pd.DataFrame(resultsDict[team][i])],axis=0)
for i in range(len(resultsDF)):
    if 10+resultsDF.iloc[i]['PTS'] < resultsDF.iloc[i]['PREV AVG PTS']:
        FLAG.append('🚩')
    else: FLAG.append(' ')
resultsDF['FLAG'] = FLAG
resultsDF[resultsDF['FLAG'] == '🚩'].sort_values(by='PLAYER')

,PTS,PREV AVG PTS,DATE,TEAM,OPPT,PTSDELTA,SPREAD,FLAG
PLAYER,,,,,,,,
Bobby Portis,2.0,12.466667,2023-11-24,MIL,WAS,3,-13.5,🚩
Damian Lillard,6.0,39.000000,2023-10-29,MIL,ATL,-17,-7.5,🚩
Damian Lillard,12.0,24.285714,2023-11-13,MIL,CHI,9,-9.5,🚩
Damian Lillard,14.0,25.550000,2023-12-11,MIL,CHI,4,-11.5,🚩
Giannis Antetokounmpo,16.0,27.333333,2023-11-01,MIL,TOR,-19,-4.5,🚩
Giannis Antetokounmpo,15.0,26.000000,2023-11-08,MIL,DET,2,-12.0,🚩
Giannis Antetokounmpo,11.0,31.400000,2023-12-19,MIL,SAS,13,-16.5,🚩
Khris Middleton,5.0,15.215686,2024-04-07,MIL,NYK,-13,-4.5,🚩
Malik Beasley,0.0,12.000000,2023-11-03,MIL,NYK,5,-5.5,🚩


In [31]:
team = 'MIN'
FLAG = []
resultsDF = pd.DataFrame()
for i in range (len(resultsDict[team])):
    resultsDF = pd.concat([resultsDF,pd.DataFrame(resultsDict[team][i])],axis=0)
for i in range(len(resultsDF)):
    if 10+resultsDF.iloc[i]['PTS'] < resultsDF.iloc[i]['PREV AVG PTS']:
        FLAG.append('🚩')
    else: FLAG.append(' ')
resultsDF['FLAG'] = FLAG
resultsDF[resultsDF['FLAG'] == '🚩'].sort_values(by='PLAYER')

,PTS,PREV AVG PTS,DATE,TEAM,OPPT,PTSDELTA,SPREAD,FLAG
PLAYER,,,,,,,,
Anthony Edwards,9.0,25.897436,2024-01-22,MIN,CHA,-3,-14.0,🚩
Anthony Edwards,11.0,26.607143,2024-03-01,MIN,SAC,-4,-8.5,🚩
Anthony Edwards,13.0,26.344828,2024-03-04,MIN,POR,5,-15.0,🚩
Anthony Edwards,9.0,26.441176,2024-03-27,MIN,DET,15,-16.5,🚩
Anthony Edwards,14.0,26.259740,2024-04-12,MIN,ATL,3,-12.5,🚩
Anthony Edwards,13.0,26.102564,2024-04-14,MIN,PHX,-19,-2.0,🚩
Jaden McDaniels,0.0,10.857143,2023-11-14,MIN,GSW,3,-3.5,🚩
Karl-Anthony Towns,10.0,21.964286,2023-12-28,MIN,DAL,8,-12.0,🚩
Karl-Anthony Towns,9.0,21.551724,2023-12-30,MIN,LAL,2,-4.5,🚩


In [32]:
team = 'NOP'
FLAG = []
resultsDF = pd.DataFrame()
for i in range (len(resultsDict[team])):
    resultsDF = pd.concat([resultsDF,pd.DataFrame(resultsDict[team][i])],axis=0)
for i in range(len(resultsDF)):
    if 10+resultsDF.iloc[i]['PTS'] < resultsDF.iloc[i]['PREV AVG PTS']:
        FLAG.append('🚩')
    else: FLAG.append(' ')
resultsDF['FLAG'] = FLAG
resultsDF[resultsDF['FLAG'] == '🚩'].sort_values(by='PLAYER')

,PTS,PREV AVG PTS,DATE,TEAM,OPPT,PTSDELTA,SPREAD,FLAG
PLAYER,,,,,,,,
Brandon Ingram,8.0,23.636364,2023-12-15,NOP,CHA,5,-9.5,🚩
CJ McCollum,13.0,23.400000,2023-11-04,NOP,ATL,-18,-2.0,🚩
CJ McCollum,2.0,18.975610,2024-02-23,NOP,MIA,-11,-2.0,🚩
CJ McCollum,8.0,18.255319,2024-03-13,NOP,CLE,-21,-6.0,🚩
Jonas Valanciunas,2.0,12.312500,2024-04-14,NOP,LAL,-16,-2.0,🚩
Jordan Hawkins,0.0,11.500000,2023-12-15,NOP,CHA,5,-9.5,🚩
Zion Williamson,12.0,23.028986,2024-04-14,NOP,LAL,-16,-2.0,🚩


In [33]:
team = 'NYK'
FLAG = []
resultsDF = pd.DataFrame()
for i in range (len(resultsDict[team])):
    resultsDF = pd.concat([resultsDF,pd.DataFrame(resultsDict[team][i])],axis=0)
for i in range(len(resultsDF)):
    if 10+resultsDF.iloc[i]['PTS'] < resultsDF.iloc[i]['PREV AVG PTS']:
        FLAG.append('🚩')
    else: FLAG.append(' ')
resultsDF['FLAG'] = FLAG
resultsDF[resultsDF['FLAG'] == '🚩'].sort_values(by='PLAYER')

,PTS,PREV AVG PTS,DATE,TEAM,OPPT,PTSDELTA,SPREAD,FLAG
PLAYER,,,,,,,,
Immanuel Quickley,5.0,16.294118,2023-11-30,NYK,DET,6,-14.5,🚩
RJ Barrett,9.0,20.500000,2023-11-26,NYK,PHX,-3,-6.5,🚩
RJ Barrett,9.0,19.588235,2023-12-13,NYK,UTA,-4,-6.0,🚩


In [34]:
team = 'OKC'
FLAG = []
resultsDF = pd.DataFrame()
for i in range (len(resultsDict[team])):
    resultsDF = pd.concat([resultsDF,pd.DataFrame(resultsDict[team][i])],axis=0)
for i in range(len(resultsDF)):
    if 10+resultsDF.iloc[i]['PTS'] < resultsDF.iloc[i]['PREV AVG PTS']:
        FLAG.append('🚩')
    else: FLAG.append(' ')
resultsDF['FLAG'] = FLAG
resultsDF[resultsDF['FLAG'] == '🚩'].sort_values(by='PLAYER')

,PTS,PREV AVG PTS,DATE,TEAM,OPPT,PTSDELTA,SPREAD,FLAG
PLAYER,,,,,,,,
Chet Holmgren,4.0,17.578947,2023-12-06,OKC,HOU,-9,-1.5,🚩
Chet Holmgren,4.0,16.978261,2024-01-29,OKC,MIN,-6,-2.5,🚩
Chet Holmgren,7.0,17.177419,2024-03-08,OKC,MIA,7,-8.5,🚩
Chet Holmgren,2.0,16.943662,2024-03-27,OKC,HOU,-6,-5.0,🚩
Chet Holmgren,5.0,16.780822,2024-03-31,OKC,NYK,1,-2.0,🚩
Jalen Williams,7.0,19.333333,2024-03-04,OKC,LAL,-12,-1.0,🚩
Shai Gilgeous-Alexander,20.0,31.237288,2024-03-04,OKC,LAL,-12,-1.0,🚩
Shai Gilgeous-Alexander,20.0,31.092308,2024-03-16,OKC,MEM,6,-10.5,🚩
Shai Gilgeous-Alexander,19.0,30.442857,2024-03-31,OKC,NYK,1,-2.0,🚩


In [35]:
team = 'ORL'
FLAG = []
resultsDF = pd.DataFrame()
for i in range (len(resultsDict[team])):
    resultsDF = pd.concat([resultsDF,pd.DataFrame(resultsDict[team][i])],axis=0)
for i in range(len(resultsDF)):
    if 10+resultsDF.iloc[i]['PTS'] < resultsDF.iloc[i]['PREV AVG PTS']:
        FLAG.append('🚩')
    else: FLAG.append(' ')
resultsDF['FLAG'] = FLAG
resultsDF[resultsDF['FLAG'] == '🚩'].sort_values(by='PLAYER')

,PTS,PREV AVG PTS,DATE,TEAM,OPPT,PTSDELTA,SPREAD,FLAG
PLAYER,,,,,,,,
Cole Anthony,4.0,15.882353,2023-12-01,ORL,WAS,5,-10.5,🚩
Jalen Suggs,0.0,12.948718,2024-01-26,ORL,MEM,-1,-6.0,🚩
Paolo Banchero,10.0,21.125000,2023-12-20,ORL,MIA,-9,-5.5,🚩


In [36]:
team = 'PHI'
FLAG = []
resultsDF = pd.DataFrame()
for i in range (len(resultsDict[team])):
    resultsDF = pd.concat([resultsDF,pd.DataFrame(resultsDict[team][i])],axis=0)
for i in range(len(resultsDF)):
    if 10+resultsDF.iloc[i]['PTS'] < resultsDF.iloc[i]['PREV AVG PTS']:
        FLAG.append('🚩')
    else: FLAG.append(' ')
resultsDF['FLAG'] = FLAG
resultsDF[resultsDF['FLAG'] == '🚩'].sort_values(by='PLAYER')

,PTS,PREV AVG PTS,DATE,TEAM,OPPT,PTSDELTA,SPREAD,FLAG
PLAYER,,,,,,,,
Cameron Payne,2.0,13.666667,2024-02-14,PHI,MIA,-5,-2.5,🚩
Tobias Harris,6.0,18.000000,2023-12-06,PHI,WAS,5,-10.5,🚩
Tobias Harris,3.0,16.800000,2023-12-18,PHI,CHI,-4,-10.5,🚩
Tobias Harris,5.0,17.303030,2024-01-05,PHI,NYK,-36,-6.0,🚩
Tobias Harris,7.0,17.729167,2024-02-22,PHI,NYK,-14,-1.5,🚩
Tyrese Maxey,16.0,26.205128,2024-01-20,PHI,CHA,8,-12.0,🚩


In [37]:
team = 'PHX'
FLAG = []
resultsDF = pd.DataFrame()
for i in range (len(resultsDict[team])):
    resultsDF = pd.concat([resultsDF,pd.DataFrame(resultsDict[team][i])],axis=0)
for i in range(len(resultsDF)):
    if 10+resultsDF.iloc[i]['PTS'] < resultsDF.iloc[i]['PREV AVG PTS']:
        FLAG.append('🚩')
    else: FLAG.append(' ')
resultsDF['FLAG'] = FLAG
resultsDF[resultsDF['FLAG'] == '🚩'].sort_values(by='PLAYER')

,PTS,PREV AVG PTS,DATE,TEAM,OPPT,PTSDELTA,SPREAD,FLAG
PLAYER,,,,,,,,
Bradley Beal,6.0,16.400000,2023-12-15,PHX,NYK,-17,-5.0,🚩
Bradley Beal,7.0,18.200000,2024-03-02,PHX,HOU,-9,-9.0,🚩
Devin Booker,8.0,29.444444,2023-11-29,PHX,TOR,-7,-3.5,🚩
Devin Booker,16.0,26.566667,2024-01-16,PHX,SAC,2,-4.5,🚩
Devin Booker,16.0,27.000000,2024-01-22,PHX,CHI,2,-5.0,🚩
Devin Booker,14.0,27.186441,2024-03-29,PHX,OKC,-25,-2.5,🚩
Devin Booker,12.0,27.312500,2024-04-09,PHX,LAC,-13,-9.5,🚩
Eric Gordon,0.0,14.037037,2023-12-31,PHX,ORL,5,-5.5,🚩
Eric Gordon,3.0,13.250000,2024-01-14,PHX,POR,11,-14.5,🚩


In [38]:
team = 'POR'
FLAG = []
resultsDF = pd.DataFrame()
for i in range (len(resultsDict[team])):
    resultsDF = pd.concat([resultsDF,pd.DataFrame(resultsDict[team][i])],axis=0)
for i in range(len(resultsDF)):
    if 10+resultsDF.iloc[i]['PTS'] < resultsDF.iloc[i]['PREV AVG PTS']:
        FLAG.append('🚩')
    else: FLAG.append(' ')
resultsDF['FLAG'] = FLAG
resultsDF[resultsDF['FLAG'] == '🚩'].sort_values(by='PLAYER')

,PTS,PREV AVG PTS,DATE,TEAM,OPPT,PTSDELTA,SPREAD,FLAG
PLAYER,,,,,,,,
Anfernee Simons,9.0,28.500000,2023-12-14,POR,UTA,-8,-5.0,🚩
Anfernee Simons,10.0,23.258065,2024-02-25,POR,CHA,-13,-3.5,🚩
Shaedon Sharpe,4.0,18.772727,2023-12-14,POR,UTA,-8,-5.0,🚩


In [39]:
team = 'SAC'
FLAG = []
resultsDF = pd.DataFrame()
for i in range (len(resultsDict[team])):
    resultsDF = pd.concat([resultsDF,pd.DataFrame(resultsDict[team][i])],axis=0)
for i in range(len(resultsDF)):
    if 10+resultsDF.iloc[i]['PTS'] < resultsDF.iloc[i]['PREV AVG PTS']:
        FLAG.append('🚩')
    else: FLAG.append(' ')
resultsDF['FLAG'] = FLAG
resultsDF[resultsDF['FLAG'] == '🚩'].sort_values(by='PLAYER')

,PTS,PREV AVG PTS,DATE,TEAM,OPPT,PTSDELTA,SPREAD,FLAG
PLAYER,,,,,,,,
De'Aaron Fox,14.0,31.857143,2023-11-20,SAC,NOP,-36,-1.5,🚩
De'Aaron Fox,15.0,29.961538,2024-01-03,SAC,ORL,3,-5.5,🚩
De'Aaron Fox,12.0,27.833333,2024-02-07,SAC,DET,-13,-13.5,🚩
Domantas Sabonis,10.0,20.833333,2023-11-20,SAC,NOP,-36,-1.5,🚩
Domantas Sabonis,9.0,19.866667,2023-11-28,SAC,GSW,1,-2.0,🚩
Harrison Barnes,10.0,33.000000,2023-10-27,SAC,GSW,-8,-3.0,🚩
Harrison Barnes,3.0,15.600000,2023-11-06,SAC,HOU,-25,-2.0,🚩
Keegan Murray,5.0,16.208333,2023-12-26,SAC,POR,-17,-9.0,🚩
Keegan Murray,5.0,16.205128,2024-01-29,SAC,MEM,9,-11.0,🚩


In [40]:
team = 'TOR'
FLAG = []
resultsDF = pd.DataFrame()
for i in range (len(resultsDict[team])):
    resultsDF = pd.concat([resultsDF,pd.DataFrame(resultsDict[team][i])],axis=0)
for i in range(len(resultsDF)):
    if 10+resultsDF.iloc[i]['PTS'] < resultsDF.iloc[i]['PREV AVG PTS']:
        FLAG.append('🚩')
    else: FLAG.append(' ')
resultsDF['FLAG'] = FLAG
resultsDF[resultsDF['FLAG'] == '🚩'].sort_values(by='PLAYER')

,PTS,PREV AVG PTS,DATE,TEAM,OPPT,PTSDELTA,SPREAD,FLAG
PLAYER,,,,,,,,
Dennis Schroder,4.0,16.666667,2023-11-13,TOR,WAS,4,-8.0,🚩
Gary Trent Jr.,0.0,11.222222,2023-12-08,TOR,CHA,-3,-5.5,🚩
Scottie Barnes,7.0,20.132075,2024-02-12,TOR,SAS,-23,-6.0,🚩


In [41]:
team = 'UTA'
FLAG = []
resultsDF = pd.DataFrame()
for i in range (len(resultsDict[team])):
    resultsDF = pd.concat([resultsDF,pd.DataFrame(resultsDict[team][i])],axis=0)
for i in range(len(resultsDF)):
    if 10+resultsDF.iloc[i]['PTS'] < resultsDF.iloc[i]['PREV AVG PTS']:
        FLAG.append('🚩')
    else: FLAG.append(' ')
resultsDF['FLAG'] = FLAG
resultsDF[resultsDF['FLAG'] == '🚩'].sort_values(by='PLAYER')

,PTS,PREV AVG PTS,DATE,TEAM,OPPT,PTSDELTA,SPREAD,FLAG
PLAYER,,,,,,,,
Collin Sexton,1.0,12.500000,2023-11-22,UTA,POR,-16,-3.5,🚩
Collin Sexton,6.0,17.478261,2024-01-29,UTA,BKN,-33,-1.0,🚩
Lauri Markkanen,13.0,23.833333,2024-01-29,UTA,BKN,-33,-1.0,🚩


In [42]:
team = 'WAS'
FLAG = []
resultsDF = pd.DataFrame()
for i in range (len(resultsDict[team])):
    resultsDF = pd.concat([resultsDF,pd.DataFrame(resultsDict[team][i])],axis=0)
for i in range(len(resultsDF)):
    if 10+resultsDF.iloc[i]['PTS'] < resultsDF.iloc[i]['PREV AVG PTS']:
        FLAG.append('🚩')
    else: FLAG.append(' ')
resultsDF['FLAG'] = FLAG
resultsDF[resultsDF['FLAG'] == '🚩'].sort_values(by='PLAYER')

,PTS,PREV AVG PTS,DATE,TEAM,OPPT,PTSDELTA,SPREAD,FLAG
PLAYER,,,,,,,,
Kyle Kuzma,11.0,22.20000,2024-01-20,WAS,SAS,-4,-2.0,🚩
Kyle Kuzma,12.0,22.38806,2024-03-29,WAS,DET,-9,-3.5,🚩


In [43]:
################################################
################################################
################################################
################################################
################################################
STOP

NameError: name 'STOP' is not defined

In [ ]:
GAMEID = str(22300104)
boxScoreDF = pd.read_csv('/Users/olutosinfashusi/jupyter/csv/boxScoreCSV/boxScoreCSVs/'+ GAMEID +'.csv')#.fillna(' ')
boxScoreDF = boxScoreDF[boxScoreDF['TEAM'] == team]
boxScoreDF.insert(1,'OPPT',teamDF.iloc[i]['OPPT'])
boxScoreDF = boxScoreDF.reset_index(drop=True).dropna()
boxScoreDF = cleanBoxScore(boxScoreDF);
boxScoreDF

In [ ]:
results = []

In [ ]:
for i in range (len(teamDF)):
    GAMEID = teamDF.iloc[i]['GAMEID']
    GAMEID = str(GAMEID)
    print(GAMEID)
    boxScoreDF = pd.read_csv('/Users/olutosinfashusi/jupyter/csv/boxScoreCSV/boxScoreCSVs/'+ GAMEID +'.csv')#.fillna(' ')
    boxScoreDF = boxScoreDF[boxScoreDF['TEAM'] == 'ATL']
    boxScoreDF.insert(1,'OPPT',teamDF.iloc[i]['OPPT'])
    boxScoreDF = boxScoreDF.reset_index(drop=True).dropna()
    boxScoreDF = cleanBoxScore(boxScoreDF);
    
    prevGamesDF = NBAdf[NBAdf['TEAM'] == 'ATL']
    prevGamesDF  = prevGamesDF[prevGamesDF['GAME DATE'] < teamDF.iloc[i]['GAME DATE']]
    prevGamesDF.reset_index(drop=True);

    prevGameIDs = prevGamesDF['GAMEID'].values
    prevOppts = prevGamesDF['OPPT'].values

    prevBoxScoresDF = pd.DataFrame()
    for prevIter in range (len(prevGameIDs)):
        GAMEID = str(prevGameIDs[prevIter])
        prevBoxScoreDF = pd.read_csv('/Users/olutosinfashusi/jupyter/csv/boxScoreCSV/boxScoreCSVs/'+ GAMEID +'.csv')#.fillna(' ')
        prevBoxScoreDF = prevBoxScoreDF[prevBoxScoreDF['TEAM'] == 'ATL']
        prevBoxScoreDF.insert(1,'OPPT',prevOppts[prevIter])
        prevBoxScoreDF = cleanBoxScore(prevBoxScoreDF)
        prevBoxScoresDF = pd.concat([prevBoxScoresDF,prevBoxScoreDF],axis=0)

    prevBoxScoresDF.groupby("PLAYER")['PTS'].mean()
    boxScoreDF.groupby("PLAYER")['PTS'].mean()
    prevAVGPTS = pd.DataFrame(prevBoxScoresDF.groupby("PLAYER")['PTS'].mean())

    prevAVGPTS = prevAVGPTS.rename(columns={'PTS':'PREV AVG PTS'})
    df = pd.concat([boxScoreDF.groupby("PLAYER")['PTS'].mean(),
           prevAVGPTS.groupby("PLAYER")['PREV AVG PTS'].mean()],axis=1).dropna()
    df['DATE'] = teamDF.iloc[i]['GAME DATE']
    df['TEAM'] = teamDF.iloc[i]['TEAM']
    df['OPPT'] = teamDF.iloc[i]['OPPT']
    results.append(df)
    pd.concat(results)

In [ ]:
pd.concat(results)

In [ ]:
#boxScoreDF

In [ ]:
prevBoxScoreDF.groupby('PLAYER')['PTS'].mean()

In [ ]:
type(prevBoxScoreDF.iloc[0]['PTS'])

In [ ]:
#pd.concat(results)

In [ ]:
prevBoxScoresDF.groupby("PLAYER")['PTS'].mean()

In [ ]:
ATLF.iloc[0].to_frame().T['GAMEID'].values[0]

In [ ]:
GAMEID = str(ATLF.iloc[0].to_frame().T['GAMEID'].values[0])


In [ ]:
pd.read_csv('/Users/olutosinfashusi/jupyter/csv/boxScoreCSV/boxScoreCSVs/'+ GAMEID +'.csv').fillna(' ')

In [ ]:
NBAdf.head(1)['GAME DATE'].values

In [ ]:
NBAdf[(NBAdf['TEAM'] == 'TOR') 
               & (NBAdf['OPPT'] == 'SAC') 
               & (NBAdf['GAME DATE'] == datetime.date(2024,3,20))]

In [ ]:
NBAdf[(NBAdf['TEAM'] == 'TOR') 
               & (NBAdf['OPPT'] == 'LAC') 
               & (NBAdf['GAME DATE'] == datetime.date(2024,1,26))]

In [ ]:
GAMEID = NBAdf[(NBAdf['TEAM'] == 'TOR') 
               & (NBAdf['OPPT'] == 'LAC') 
               & (NBAdf['GAME DATE'] == datetime.date(2024,1,26))]['GAMEID'].values[0]
GAMEID = str(GAMEID)
pd.read_csv('/Users/olutosinfashusi/jupyter/csv/boxScoreCSV/boxScoreCSVs/'+GAMEID +'.csv').fillna(' ')

In [ ]:
#irisDF[~(irisDF['petal_width'] <= 1.75) & (irisDF['petal_length'] <= 4.85)]
GAMEID = NBAdf[(NBAdf['TEAM'] == 'TOR') 
               & (NBAdf['OPPT'] == 'SAC') 
               & (NBAdf['GAME DATE'] == datetime.date(2024,3,20))]['GAMEID'].values[0]
GAMEID = str(GAMEID)
print(GAMEID)

In [ ]:
pd.read_csv('/Users/olutosinfashusi/jupyter/csv/boxScoreCSV/boxScoreCSVs/'+GAMEID +'.csv').fillna(' ')